In [1]:
import pandas as pd; import os; from analysis_helpers import *; import warnings
warnings.filterwarnings('ignore')

In [55]:
def sum_pd(subdir):
    '''
    input: subject directory (string)
    output: full experiment info (dataframe)
    '''

    files = [ x for x in os.listdir(subdir) if 'pres' in x or 'mem' in x ]
    df_list = [ pd.read_csv(subdir+'/'+x) for x in files ]
    df = pd.concat(df_list, ignore_index=True)

    return(df)

# Functions for Simple Behavioral Analyses

def add_level(df):
    '''
    input: subject dataframe
    output: subject dataframe w/ Attention Level string for each Memory trial row
    '''
    for x in df.Run.unique():
        mask = df['Run']==x
        df[mask] = run_level(df[mask])

    return(df)

def run_level(df):
    '''
    input: df containing pres and mem from single run
    output: df with string in 'Attention Level' column in each Memory trial row
    '''
    #cued_cat = df[df['Trial Type']=='Presentation']['Cued Category'].tolist()[0]

    for index,row in df.iterrows():

        if row['Trial Type']=='Memory':
            mem_image = row['Memory Image']
            for cue in ['Cued ', 'Uncued ']:
                for cat in ['Face', 'Place']:
                    if df.loc[df[cue+cat] == mem_image].shape[0]!=0:
                        cued_cat = df.loc[df[cue+cat] == mem_image]['Cued Category'].item()
                        if cat == cued_cat:
                            df['Category'][index]=cued_cat
                            if cue == 'Cued ':
                                attention = "Full"
                            elif cue == 'Uncued ':
                                attention = "Category"
                        else:
                            df['Category'][index]=cat
                            if cue == 'Uncued ':
                                attention = "None"
                            elif cue == 'Cued ':
                                attention = "Side"
                        df['Attention Level'][index] = attention
                        df['Cued Category'][index] = cued_cat

    mem_mask = df['Trial Type']=='Memory'
    df.loc[mem_mask,'Attention Level'] = df.loc[mem_mask,'Attention Level'].fillna('Novel')

    return(df)

In [56]:
df.loc[df[cue+cat] == mem_image]['Cued Category'][0]

NameError: name 'cue' is not defined

In [57]:
sub_list = []

data_dir = '/Users/kirstenziman/Documents/attention-memory-task/data'

for sub_dir in os.listdir(data_dir):
    print(sub_dir)
    sub_list.append(add_level(sum_pd(data_dir + '/' + sub_dir)))

8_2019_Feb_28
15_2019_Apr_01
1_2019_Feb_21
25_2019_Apr_16
12_2019_Mar_27
4_2019_Feb_21
28_2019_Apr_23
19_2019_Apr_01
21_2019_Apr_09
33_2019_Apr_26
7_2019_Feb_28
26_2019_Apr_16
14_2019_Mar_29
2_2019_Feb_21
16_2019_Apr_01
27_2019_Apr_22
23_2019_Apr_10
18_2019_Apr_01
29_2019_Apr_23
13_2019_Mar_29
5_2019_Feb_26
24_2019_Apr_11
0_2019_Feb_21
34_2019_Apr_27
22_2019_Apr_10
327_2019_Mar_27
17_2019_Apr_01
3_2019_Feb_21
10_2019_Mar_27
20_2019_Apr_09


In [20]:
df = pd.concat(sub_list)

In [22]:
df.to_csv('raw_behav_from_add_level.csv')
exp2 = df

In [23]:
exp2.ix[ exp2['Memory Image'].str.contains("sun", na=False),'Category']='Place'
exp2.ix[~exp2['Memory Image'].str.contains("sun", na=False),'Category']='Face'

In [25]:
exp2['Last Cued'] = np.nan

for sub in exp2['Subject'].unique():
    for run in exp2['Run'].unique():
        
        # obtain cued category from the last presentation trial
        last_cat = exp2[(exp2['Trial Type']=='Presentation')
                       & (exp2['Subject']==sub)
                       & (exp2['Run']==run)]['Cued Category'].iloc[-1]
        
        # obtain cued category from the last run
        exp2.loc[(exp2['Trial Type']=='Memory')
                       & (exp2['Subject']==sub)
                       & (exp2['Run']==run),'Last Cued'] = last_cat

In [30]:
exp2[exp2['Last Cued']=='Place'].shape

(5480, 27)

In [31]:
exp2[exp2['Last Cued']=='Face'].shape

(4120, 27)

In [39]:
exp2[(exp2['Trial Type']=='Presentation')
                       & (exp2['Subject']==sub)
                       & (exp2['Run']==run)]['Cued Category']#.iloc[-6]

70    Place
71    Place
72    Place
73     Face
74    Place
75     Face
76     Face
77    Place
78     Face
79     Face
Name: Cued Category, dtype: object

In [52]:
tr.to_csv('~/Desktop/try_this_one.csv')

In [42]:
exp2.ix[ exp2['Memory Image'].str.contains("sun", na=False),'Category']='Place'
exp2.ix[~exp2['Memory Image'].str.contains("sun", na=False),'Category']='Face'

In [44]:
def add_nov_label(combo):
    '''
    input:  dataframe of participant data, and the name of the column to use for cue info
            for exp1 use 'Cued Category' (cue for that block)
            for exp2 use 'Last Cued'     (last cue from presentation block)
    output: dataframe where novel images are labeled by cued or uncued category
    '''

    for cat in ['Face','Place']:

        # for all 'Novel_Uncued', if image in cued category, rename 'Novel_Cued'
        combo.loc[(combo['Trial Type']=='Memory') 
                  & (combo['Last Cued']==cat) 
                  & (combo['Category']==cat)
                  & (combo['Attention Level']=='Novel'),
                     'Attention Level'] = 'Nov_Cued'
        
        combo.loc[(combo['Trial Type']=='Memory') 
                  & (combo['Last Cued']==cat) 
                  & (combo['Category']!=cat)
                  & (combo['Attention Level']=='Novel'),
                     'Attention Level'] = 'Nov_Un'
    return(combo)

In [48]:
l = []

for sub in exp2['Subject'].unique():
    print(sub)
    a = add_nov_label(exp2[exp2['Subject']==sub])
    l.append(a)

8
15
1
25
12
4
28
19
21
33
7
26
14
2
16
27
23
18
29
13
5
24
0
34
22
327
17
3
10
20


In [50]:
tr = pd.concat(l)

In [54]:
tr['Attention Level']

0           NaN
1           NaN
2           NaN
3           NaN
4           NaN
5           NaN
6           NaN
7           NaN
8           NaN
9           NaN
10          NaN
11          NaN
12          NaN
13          NaN
14          NaN
15          NaN
16          NaN
17          NaN
18          NaN
19          NaN
20          NaN
21          NaN
22          NaN
23          NaN
24          NaN
25          NaN
26          NaN
27          NaN
28          NaN
29          NaN
         ...   
370       Place
371        Face
372       Place
373      Nov_Un
374      Nov_Un
375        Face
376       Place
377    Nov_Cued
378      Nov_Un
379    Nov_Cued
380        Face
381      Nov_Un
382    Nov_Cued
383    Nov_Cued
384        Face
385    Nov_Cued
386        Face
387      Nov_Un
388       Place
389    Nov_Cued
390       Place
391      Nov_Un
392       Place
393        Face
394        Face
395      Nov_Un
396        Face
397    Nov_Cued
398       Place
399    Nov_Cued
Name: Attention Level, L